In [1]:
import io
import time
import torch
import numpy as np
import PIL
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc
from compressai.zoo import cheng2020_anchor
from piq import LPIPS, DISTS, psnr, multi_scale_ssim

class Config: pass

In [5]:
codec = cheng2020_anchor(quality=6, metric='mse', pretrained=True)
codec.eval();

In [6]:
lpips_loss = LPIPS().to("cuda")
dists_loss = DISTS().to("cuda")

/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
dataset = load_dataset("danjacobellis/LSDIR", split='validation')

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

In [61]:
def DGML_compress(sample):
    device = "cuda"
    codec = codec.to(device)
    sample = dataset[28]
    with torch.no_grad():
        img = sample['image']
        x = PILToTensor()(img).to(torch.float)
        x = (x/255).unsqueeze(0).to(device)
        H, W = x.size(2), x.size(3)
        x_padded = walloc.pad(x,p=16)
    
        t0 = time.time()
        Y = codec.g_a(x_padded).round()
        encode_time = time.time() - t0
    
        t0 = time.time()
        x_hat = codec.g_s(Y)
        x_hat = x_hat.clamp(0,1)
        decode_time = time.time() - t0
        
        x_hat = walloc.crop(x_hat, (H,W))
        rec = ToPILImage()(x_hat[0])
        buff2 = io.BytesIO()
        rec.save(buff2, format='png', lossless=True)
        rec_png_bytes = buff2.getbuffer()
    
        bpp = 1.
        PSNR = psnr(x,x_hat)
        MSSIM = multi_scale_ssim(x,x_hat)
        LPIPS_dB = -10*np.log10(lpips_loss(x.to("cuda"), x_hat.to("cuda")).item())
        DISTS_dB = -10*np.log10(dists_loss(x.to("cuda"), x_hat.to("cuda")).item())        
        
        return {
            'recovered': rec_png_bytes,
            'encode_time': encode_time,
            'decode_time': decode_time,
            'bpp': bpp,
            'PSNR': PSNR,
            'MSSIM': MSSIM,
            'LPIPS_dB': LPIPS_dB,
            'DISTS_dB': DISTS_dB,
        }